In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
class Seq2Seq(nn.Module):
    def __init__(self, sequence_length, num_features, static_dim, output_dim):
        super(Seq2Seq, self).__init__()
        self.encoder = nn.GRU(num_features, 128, batch_first=True) #128 adalah hidden_dim
        self.decoder = nn.GRU(num_features, 128, batch_first=True)
        self.fc1 = nn.Linear(sequence_length * 128 + static_dim, 64)  # For regression, use the entire sequence
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.fc4 = nn.Linear(sequence_length * 128, 64)  # For classification, use the entire sequence

    def forward(self, timeseries_input, static_input):
        _, hidden = self.encoder(timeseries_input)
        decoder_input = torch.zeros_like(timeseries_input)  # Initialize decoder input with zeros
        gru_output_seq, _ = self.decoder(decoder_input, hidden)

        # Use the entire sequence for regression
        gru_output_seq_flat = gru_output_seq.contiguous().view(gru_output_seq.size(0), -1)
        concatenated = torch.cat((gru_output_seq_flat, static_input), dim=1)
        x = torch.relu(self.fc1(concatenated))
        x = torch.relu(self.fc2(x))
        regression = self.fc3(x)

        # Use the entire sequence for classification
        # y = torch.relu(self.fc4(gru_output_seq_flat))
        # y = torch.relu(self.fc2(y))
        # classif = torch.sigmoid(self.fc3(y))

        return regression

In [3]:
sequence_length = 10
num_features = 5  # number of features in the input sequence
static_dim = 3
output_dim = 1  # output dimension for regression
batch_size = 32
num_epochs = 10



In [5]:
def generate_data(num_samples, sequence_length, num_features, static_dim):
    timeseries_data = np.random.rand(num_samples, sequence_length, num_features)
    static_data = np.random.rand(num_samples, static_dim)
    regression_labels = np.random.rand(num_samples, output_dim)  # Continuous labels for regression
    #classification_labels = np.random.randint(0, 2, size=(num_samples, 1))  # Binary labels for classification
    return timeseries_data, static_data, regression_labels

num_samples = 1000
timeseries_data, static_data, regression_labels= generate_data(num_samples, sequence_length, num_features, static_dim)

In [6]:
timeseries_data = torch.tensor(timeseries_data, dtype=torch.float32)
static_data = torch.tensor(static_data, dtype=torch.float32)
regression_labels = torch.tensor(regression_labels, dtype=torch.float32)


train_size = int(0.8 * num_samples)
train_dataset = TensorDataset(timeseries_data[:train_size], static_data[:train_size], regression_labels[:train_size])
val_dataset = TensorDataset(timeseries_data[train_size:], static_data[train_size:], regression_labels[train_size:])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [7]:
model = Seq2Seq(sequence_length, num_features, static_dim, output_dim)
regression_criterion = nn.MSELoss()
classification_criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
for epoch in range(num_epochs):
    model.train()
    for timeseries_batch, static_batch, regression_labels_batch in train_loader:
        optimizer.zero_grad()
        regression_outputs = model(timeseries_batch, static_batch)
        regression_loss = regression_criterion(regression_outputs, regression_labels_batch)
        loss = regression_loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for timeseries_batch, static_batch, regression_labels_batch in val_loader:
            regression_outputs = model(timeseries_batch, static_batch)
            regression_loss = regression_criterion(regression_outputs, regression_labels_batch)
            loss = regression_loss 
            val_loss += loss.item()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

Epoch 1/10, Validation Loss: 0.0869
Epoch 2/10, Validation Loss: 0.0894
Epoch 3/10, Validation Loss: 0.0840
Epoch 4/10, Validation Loss: 0.0857
Epoch 5/10, Validation Loss: 0.0823
Epoch 6/10, Validation Loss: 0.0866
Epoch 7/10, Validation Loss: 0.0828
Epoch 8/10, Validation Loss: 0.0882
Epoch 9/10, Validation Loss: 0.0883
Epoch 10/10, Validation Loss: 0.0826
